In [ ]:
import pandas as pd

In [ ]:
helmet_pd = pd.read_csv("../input/nfl-health-and-safety-helmet-assignment/image_labels.csv")
helmet_pd.head()

In [ ]:
!mkdir -p /kaggle/working/images/train
!mkdir -p /kaggle/working/labels/train
!mkdir -p /kaggle/working/images/valid
!mkdir -p /kaggle/working/labels/valid
!mkdir -p /kaggle/working/yolov5_weight

In [ ]:
list_images = helmet_pd['image'].unique()
len(list_images)

In [ ]:
from sklearn.model_selection import train_test_split

lst_train, lst_valid = train_test_split(list_images, test_size=0.1, random_state=42)
print(len(lst_train))
print(len(lst_valid))

In [ ]:
W, H = 1280, 720
def create_dataset(list_images, data_type):
    for image in list_images:
        df = helmet_pd[helmet_pd["image"]==image]
        file_name = image.split(".")[0] + ".txt"
        content = []
        for index, row in df.iterrows():
            center_x = (row['left'] + (row['width'] / 2)) / W
            center_y = (row['top'] + (row['height'] / 2)) / H
            box_w = row['width'] / W
            box_h = row['height'] / H
            row = " ".join(["0", str(center_x), str(center_y), str(box_w), str(box_h)])
            content.append(row)
        file = open("/kaggle/working/labels/" + data_type + "/" + file_name, "w")
        file.write("\n".join(content))
        file.close()

In [ ]:
create_dataset(lst_train, "train")
create_dataset(lst_valid, "valid")

In [ ]:
from shutil import copyfile
import os

def copy_images(lst_images, dst_dir):
    for img in lst_images:
        src = os.path.join('/kaggle/input/nfl-health-and-safety-helmet-assignment/images/', img)
        dst = os.path.join(dst_dir, img) 
        copyfile(src, dst)

In [ ]:
copy_images(lst_train, "/kaggle/working/images/train")
copy_images(lst_valid, "/kaggle/working/images/valid")

### Preview one image

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import random

file_name = random.choice(lst_train).split(".")[0]

label_path = "/kaggle/working/labels/train/"+file_name+".txt"
fl = open(label_path, 'r')
data = fl.read().split("\n")
fl.close()

im = Image.open('/kaggle/working/images/train/'+file_name+'.jpg')
# Create figure and axes
fig, ax = plt.subplots()
fig.set_figheight(15)
fig.set_figwidth(15)
# Display the image
ax.imshow(im)

for dt in data:
    dt = dt.strip()
    li, x, y, w, h, = map(float, dt.split(' '))
    
    l = int((x - w / 2) * W)
    r = int((x + w / 2) * W)
    t = int((y - h / 2) * H)
    b = int((y + h / 2) * H)

    # Create a Rectangle patch
    rect = patches.Rectangle((l, t), r-l, b-t, linewidth=1, edgecolor='r', facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)

plt.show()


In [ ]:
%cd /kaggle/working
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git checkout v6.0
!pip install -r requirements.txt

In [ ]:
%%writefile nfl.yaml

train: /kaggle/working/images/train
val: /kaggle/working/images/valid

# number of classes
nc: 1

# class names
names: ['Helmet']

In [ ]:
!WANDB_MODE="disabled" python train.py --img 1280 --batch 8 --epochs 20 --data nfl.yaml --weights yolov5m6.pt 

In [ ]:
!cp -r runs/train/exp/ /kaggle/working/yolov5_weight
!rm -rf /kaggle/working/images
!rm -rf /kaggle/working/labels
!rm -rf /kaggle/working/yolov5